# Backtesting Trading Strategy

This notebook is used for backtesting the trading strategy based on the generated signals from the AI-powered Bitcoin trade signal generation system.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.inference.signal_generator import SignalGenerator
from src.data.collector import collect_historical_data

# Load historical data
data = collect_historical_data('BTC/USDT', '1m', '2021-01-01', '2023-01-01')

# Initialize the signal generator
signal_generator = SignalGenerator()

# Generate signals
signals = signal_generator.generate_signals(data)

# Backtesting function
def backtest_strategy(data, signals):
    initial_balance = 1000  # Starting with $1000
    balance = initial_balance
    position = 0  # Current position in BTC
    trade_history = []

    for i in range(len(data)):
        if signals[i] == 1:  # Buy signal
            position = balance / data['close'][i]
            balance = 0
            trade_history.append(('BUY', data['date'][i], data['close'][i]))
        elif signals[i] == -1 and position > 0:  # Sell signal
            balance = position * data['close'][i]
            position = 0
            trade_history.append(('SELL', data['date'][i], data['close'][i]))

    final_balance = balance + (position * data['close'].iloc[-1])
    return final_balance, trade_history

# Run backtest
final_balance, trade_history = backtest_strategy(data, signals)

# Results
print(f'Initial Balance: $1000')
print(f'Final Balance: ${final_balance:.2f}')
print('Trade History:')
for trade in trade_history:
    print(trade)

# Plotting results
plt.figure(figsize=(14, 7))
plt.plot(data['date'], data['close'], label='BTC Price')
plt.title('BTC Price and Trades')
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.show()